In [1]:
import pandas as pd
import tabula

columns = ['state', 'river-basin', 'river', 'no-of-stations', '2016-IKA/WQI', '2016-category', '2016-class', '2017-IKA/WQI', '2017-category', '2017-class']
def extract_table(df:pd.DataFrame) -> pd.DataFrame:
    # forward fill only first column
    df.iloc[:,[0,1]] = df.iloc[:,[0,1]].ffill()
    # replace \r with space
    df = df.replace('\r','', regex=True)
    # find the first row of first column with value 'NEGERI/STATE'
    row = df[df.iloc[:,0] == 'NEGERI/STATE'].index[0]
    # make row 0 as header
    df.columns = columns # range(df.shape[1])
    # drop row 0 & 1
    df = df.drop(df.index[range(row)])
    # reset index
    df = df.reset_index(drop=True)
    return df

In [2]:
pages = range(10,16) # jadual 2.2 : sederhana tercemar
dfs = []
for page in pages:
    df = tabula.read_pdf('Kualiti-Air-Sungai.pdf', pages=[page],stream=False,lattice=True)[0]
    dfs.append(extract_table(df))
df = pd.concat(dfs).reset_index(drop=True)

In [3]:
# remove rows with column 1 value equal to 'LEMBANGANSUNGAI/RIVER BASIN' from df
df = df[df.iloc[:,1] != 'LEMBANGANSUNGAI/RIVER BASIN'].reset_index(drop=True)
# replace with null value column 0 with value 'NEGERI/STATE'
df.iloc[:,0] = df.iloc[:,0].replace('NEGERI/STATE', None)
# forward fill column 0
df.iloc[:,0] = df.iloc[:,0].ffill()
# only in case column 9 is null, move values in column 0 to column 8 to column 1 to column 9 but maintain if column 9 is not null
mask = df.iloc[:,9].isnull()
df[mask] = df[mask].shift(axis=1)
mask = df.iloc[:,9].isnull()
df[mask] = df[mask].shift(axis=1)
df.iloc[:,[0,1]] = df.iloc[:,[0,1]].ffill()

In [4]:
df.describe()

,state,river-basin,river,no-of-stations,2016-IKA/WQI,2016-category,2016-class,2017-IKA/WQI,2017-category,2017-class
count,207,207,207,207,207,207,207,207,207,207
unique,21,88,204,9,32,3,2,21,1,2
top,Sarawak,Sg. Klang,Sg. Rasau,1,79,ST/SP,III,79,ST/SP,III
freq,40,12,2,122,18,176,120,31,207,120


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   state           207 non-null    object
 1   river-basin     207 non-null    object
 2   river           207 non-null    object
 3   no-of-stations  207 non-null    object
 4   2016-IKA/WQI    207 non-null    object
 5   2016-category   207 non-null    object
 6   2016-class      207 non-null    object
 7   2017-IKA/WQI    207 non-null    object
 8   2017-category   207 non-null    object
 9   2017-class      207 non-null    object
dtypes: object(10)
memory usage: 16.3+ KB


In [6]:
df.to_excel('Jadual-2-2-sungai-sederhana-tercemar-2017.xlsx', index=False)